# Physics informed neural networks to solve differential equations

In [ ]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

## Basic ordinary differential equation

## Lapace equation 

## Heat equation